<a href="https://colab.research.google.com/github/Nandinishyam/Machine_learning/blob/main/email_spam_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras-tuner


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [18]:
from keras.layers import Bidirectional, LSTM
from keras_tuner import HyperModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras_tuner.tuners import RandomSearch
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder

In [11]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
data=pd.read_csv('email_spam.csv')

In [6]:
data.head()

,title,text,type
0,?? the secrets to SUCCESS,"Hi James,\n\nHave you claim your complimentary...",spam
1,?? You Earned 500 GCLoot Points,"\nalt_text\nCongratulations, you just earned\n...",not spam
2,?? Your GitHub launch code,"Here's your GitHub launch code, @Mortyj420!\n ...",not spam
3,[The Virtual Reward Center] Re: ** Clarifications,"Hello,\n \nThank you for contacting the Virtua...",not spam
4,"10-1 MLB Expert Inside, Plus Everything You Ne...","Hey Prachanda Rawal,\n\nToday's newsletter is ...",spam


In [9]:
def preprocess(text):
  text=text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  text = " ".join([word for word in text.split() if word not in stopwords.words('english')])
  return text

In [26]:
data['text']=data['text'].apply(preprocess)
data['title']=data['title'].apply(preprocess)
data['content']=data['text']+''+data['title']

In [27]:
label_encoder = LabelEncoder()
data['type'] = label_encoder.fit_transform(data['type'])

In [28]:
X = data['content']
y = data['type']


In [29]:
max_words = 5000  # Maximum number of words to consider
max_len = 200     # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(sequences, maxlen=max_len)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [31]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=True),
    GlobalMaxPooling1D(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [32]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [37]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step - accuracy: 0.7510 - loss: 0.6231 - val_accuracy: 0.5714 - val_loss: 0.6995
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.7301 - loss: 0.6110 - val_accuracy: 0.5714 - val_loss: 0.7121
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.7614 - loss: 0.5654 - val_accuracy: 0.5714 - val_loss: 0.7361
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.7301 - loss: 0.5691 - val_accuracy: 0.5714 - val_loss: 0.7597


In [38]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6471 - loss: 0.6739
Test Loss: 0.6739
Test Accuracy: 0.6471


In [44]:
user_input = input("Enter your email content: ")
processed_input = preprocess(user_input)

# Tokenize and pad the input
sequence = tokenizer.texts_to_sequences([processed_input])
padded_sequence = pad_sequences(sequence, maxlen=200)  # Use the same max_len as during training

prediction = model.predict(padded_sequence)
label = "spam" if prediction[0][0] > 0.5 else "not spam"

# Output the result
print(f"The email is classified as: {label}")

Enter your email content: get complimentary tickets
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
The email is classified as: not spam
